# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [8]:
!pip install -U efficientnet

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [9]:
import numpy as np
import os
#from scipy.misc import imread, imresize
#import imread, imresize
from skimage.io import imread
from skimage.transform import resize as imresize
import datetime
import os
from tensorflow.keras.optimizers import Adam

We set the random seed so that the results don't vary drastically.

In [10]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [11]:
train_doc = np.random.permutation(open('datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('datasets/Project_data/val.csv').readlines())
batch_size = 10 #experiment with the batch size

In [12]:
#math.floor(len(train_doc) / batch_size)
#round(((len(train_doc) / batch_size)-num_batches*batch_size)
#len(train_doc)
len(train_doc) % batch_size


3

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [13]:
import math
batch_size = 64 #experiment with the batch size 
height=128 
width=128
INPUT_SIZE_IN_BATCH=10
INPUT_PER_FOLDER=30
y=height
z=width
# folder_list = train_doc/val_doc
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = 10 #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = math.floor(len(folder_list) / batch_size) # calculate the number of batches (number of videos / batch size)
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,img_idx,128,128,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            mandatory = [0,1,28,29]
            random_list_1 = rn.sample(range(2,28),10 - 4)
            random_list_1.extend(mandatory)
            random_list_1.sort()
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(random_list_1): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    image = imresize(image, (128,128),anti_aliasing=True)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = (image[:,:,0] - np.min(image[:,:,0])) / (np.max(image[:,:,0]) - np.min(image[:,:,0]))#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1] - np.min(image[:,:,1])) / (np.max(image[:,:,1]) - np.min(image[:,:,1]))#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2] - np.min(image[:,:,2])) / (np.max(image[:,:,2]) - np.min(image[:,:,2]))#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                #batch_labels[folder, int([folder + (batch*batch_size)].strip().split(';')[2])] = 1                                                                       
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        
def normalize(imag):    
    return (imag[:,:,0]-imag[:,:,0].mean())/imag[:,:,0].std(), (imag[:,:,1]-imag[:,:,1].mean())/imag[:,:,1].std(), (imag[:,:,2]-imag[:,:,2].mean())/imag[:,:,2].std()

def cropAndResize(imag,height,width):
    #crop the images and resize them. Note that the images are of 2 different shape 
    #and the conv3D will throw error if the inputs in a batch have different shapes
    # CROPPING (making aspect ratio same)
    if abs(imag.shape[0]-imag.shape[1])%2==0 and imag.shape[0]!=imag.shape[1]:
        dimension_diff=abs(imag.shape[0]-imag.shape[1])
        cropping_ratio=dimension_diff//2
        if imag.shape[0]>imag.shape[1]:
            imag=imag[cropping_ratio:imag.shape[0]-cropping_ratio,:,:]
        elif imag.shape[0]<imag.shape[1]:
            imag=imag[:,cropping_ratio:imag.shape[1]-cropping_ratio,:]
                    
    # RESIZING
    if imag.shape[0]>height or imag.shape[1]>width:
        imag=imresize(imag,size=(height,width),
                      interp='bilinear',mode='RGB')
        
    return imag


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [14]:
curr_dt_time = datetime.datetime.now()
train_path = 'datasets/Project_data/train'
val_path = 'datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [17]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.applications.vgg16 import VGG16
from keras.layers import Dropout, TimeDistributed, CuDNNLSTM, Dense
from tensorflow.keras.applications import ResNet50
import efficientnet.tfkeras as efn
#write your model here
base_model = efn.EfficientNetB0(include_top=False, weights='imagenet',input_shape=(128, 128,3))
x = base_model.output
x = Flatten()(x)

for layer in base_model.layers:
    layer.trainable = False

model = Model(inputs=base_model.input, outputs=x)

rnn_model = Sequential()
rnn_model.add(TimeDistributed(model, input_shape=(10,128,128,3)))
rnn_model.add(CuDNNLSTM(64,return_sequences=False))
rnn_model.add(Dropout(0.5))
rnn_model.add(Dense(5, activation='softmax'))

16818176/16804768 [==============================] - 1s 0us/step


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [18]:
optimiser = Adam(lr=0.001)#write your optimizer
rnn_model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 stem_conv (Conv2D)             (None, 64, 64, 32)   864         ['input_3[0][0]']                
                                                                                                  
 stem_bn (BatchNormalization)   (None, 64, 64, 32)   128         ['stem_conv[0][0]']              
                                                                                                  
 stem_activation (Activation)   (None, 64, 64, 32)   0           ['stem_bn[0][0]']            

/usr/local/lib/python3.8/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [19]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [20]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'abcd.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=False, save_weights_only=True, mode='auto', save_freq=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0.000001) # write the REducelronplateau code here
ES = EarlyStopping(monitor='val_categorical_accuracy', min_delta=0, patience=10, verbose=1, mode='auto', baseline=None, restore_best_weights=True)
callbacks_list = [checkpoint, LR, ES]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [21]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [22]:
rnn_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/tmp/ipykernel_779/2952588884.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  rnn_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  datasets/Project_data/train ; batch size = 64
Epoch 1/10


2022-11-23 17:05:55.591021: I tensorflow/stream_executor/cuda/cuda_dnn.cc:377] Loaded cuDNN version 8302



Epoch 00001: saving model to model_init_2022-11-2317_03_29.990254/abcd.h5
 1/11 [=>............................] - ETA: 1:29 - loss: 1.7311 - categorical_accuracy: 0.2031
Epoch 00001: saving model to model_init_2022-11-2317_03_29.990254/abcd.h5
 2/11 [====>.........................] - ETA: 22s - loss: 1.8086 - categorical_accuracy: 0.2188 
Epoch 00001: saving model to model_init_2022-11-2317_03_29.990254/abcd.h5
 3/11 [=======>......................] - ETA: 26s - loss: 1.7155 - categorical_accuracy: 0.2552
Epoch 00001: saving model to model_init_2022-11-2317_03_29.990254/abcd.h5
 4/11 [=========>....................] - ETA: 25s - loss: 1.6426 - categorical_accuracy: 0.2930
Epoch 00001: saving model to model_init_2022-11-2317_03_29.990254/abcd.h5
 5/11 [============>.................] - ETA: 23s - loss: 1.5501 - categorical_accuracy: 0.3531
Epoch 00001: saving model to model_init_2022-11-2317_03_29.990254/abcd.h5
 6/11 [===============>..............] - ETA: 20s - loss: 1.4745 - catego